In [24]:
pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [29]:
import cv2
import os
import numpy as np

# 이미지가 있는 원본 폴더 경로
input_folder = r'C:\Users\이영진\Desktop\AI 실습\schoolstudy\PKNU_AI-programing\testing(infrared)\1~20%'

# 처리된 이미지를 저장할 상위 폴더 경로
output_folder = r'C:\Users\이영진\Desktop\AI 실습\schoolstudy\PKNU_AI-programing\infrared_processed_images'

# 상위 폴더가 없다면 생성
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 폴더 내 모든 이미지 파일 읽기
image_files = [f for f in os.listdir(input_folder) if f.endswith('.jpg') or f.endswith('.png')]

# 총 이미지 파일 수
total_files = len(image_files)

# 각 구간의 이미지 개수 계산
interval_size = total_files // 5  # 5개의 구간

# 각 구간을 처리
for i in range(5):
    # 구간별 하위 폴더 이름 (20% 단위)
    section_folder = os.path.join(output_folder, f'{i * 20 + 1}~{(i + 1) * 20}%')
    label_file_path = os.path.join(section_folder, 'label.txt')  # 라벨 파일 경로
    
    # 하위 폴더가 없다면 생성
    if not os.path.exists(section_folder):
        os.makedirs(section_folder)
    
    # 라벨 정보 작성 (현재 구간)
    label = f'{i * 20 + 1}~{(i + 1) * 20}%'
    
    # 구간의 시작 및 끝 인덱스 계산
    start_idx = i * interval_size
    end_idx = (i + 1) * interval_size if i < 4 else total_files  # 마지막 구간은 나머지 파일 포함

    # 현재 구간에 해당하는 파일들 처리
    for filename in image_files[start_idx:end_idx]:
        input_image_path = os.path.join(input_folder, filename)

        # 이미지를 읽어들임
        image = cv2.imread(input_image_path)
        
        if image is None:
            print(f"Error loading image: {filename}")
            continue
        
        # 빨간색 채널 강조하여 적외선 효과 적용
        red_channel = image[:, :, 2]  # BGR에서 빨간색 채널은 2번 인덱스
        image[:, :, 1] = 0  # 녹색 채널을 0으로 설정
        image[:, :, 0] = 0  # 파란색 채널을 0으로 설정
        
        # 그레이스케일로 변환하여 흑백 적외선 이미지 생성
        infrared_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # 새로운 파일 경로 (구간별 폴더에 이미지 이름 유지)
        output_image_path = os.path.join(section_folder, filename)
        
        # 변환된 이미지를 저장
        cv2.imwrite(output_image_path, infrared_image)
        
        print(f"Processed {filename} (Section {label}) and saved to {output_image_path}")

    # 라벨 정보를 텍스트 파일로 저장
    with open(label_file_path, 'w') as label_file:
        label_file.write(label)
    print(f"Label for Section {label} saved to {label_file_path}")

print("All images processed and labeled.")


Error loading image: Clear_Sky.A_img130.png.25nru0uj.ingestion-7c6b6d888b-6c6rr.jpg
Error loading image: Clear_Sky.A_img138.png.25nru115.ingestion-7c6b6d888b-pchsf.jpg
Error loading image: Clear_Sky.A_img139.png.25nru11a.ingestion-7c6b6d888b-6c6rr.jpg
Error loading image: Clear_Sky.A_img155.png.25nru18q.ingestion-7c6b6d888b-fctbl.jpg
Error loading image: Clear_Sky.A_img156.png.25nru18t.ingestion-7c6b6d888b-pchsf.jpg
Error loading image: Clear_Sky.A_img164.png.25nru1br.ingestion-7c6b6d888b-fctbl.jpg
Error loading image: Clear_Sky.A_img173.png.25nru1hl.ingestion-7c6b6d888b-fctbl.jpg
Error loading image: Clear_Sky.A_img175.png.25nru1ii.ingestion-7c6b6d888b-6c6rr.jpg
Label for Section 1~20% saved to C:\Users\이영진\Desktop\AI 실습\schoolstudy\PKNU_AI-programing\infrared_processed_images\1~20%\label.txt
Error loading image: Clear_Sky.A_img193.png.25nru1qo.ingestion-7c6b6d888b-6c6rr.jpg
Error loading image: Clear_Sky.A_img200.png.25nru1s1.ingestion-7c6b6d888b-fctbl.jpg
Error loading image: Clear_